In [1]:
import pyodbc
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
path = r"C:\Users\jshernandezm\genommalabinternacional\Cristian Javier Sanchez Yepez - NoB2B\Alkosto"
name_file = '\\reportesCKPRnewfilesINF_PROVEEDORESVDR0014997_20210401-20210404_20210405MER02-01.csv'

date = datetime.strptime(name_file.split('_')[2].split('-')[1], '%Y%m%d')

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

query_week = "SELECT TmpSemanaAnioGenomma FROM Gnm_DWH.dbo.Dim_Tiempo WHERE TmpFecha = '{0}'"

In [4]:
week = pd.read_sql(query_week.format(date.strftime('%Y-%m-%d')), conn)

In [5]:
df = pd.read_csv(path + name_file, encoding='latin1')

In [6]:
def clean_data(df, date, week):
    data = df.copy()
    data['EAN'] = data['EAN'].apply(lambda x: int(x.replace('=','').replace('"','')))
    data['Anio'] = date.year
    data['Semana'] = week.loc[0].values[0]
    data['Fecha Inicio'] = date - timedelta(days=6)
    data['Fecha Fin'] = date
    data['Cliente'] = 'Alkosto'
    data.rename({'UNIDAD NEGOCIO INVENTARIO':'Cod_Local', 'DESCRIPCIÓN':'Descripcion_Prod', 'INVENTARIO TOTAL':'Inventario', 'UNIDADES VENDIDAS':'Unidades', 'PRECIO + IVA':'Precio + IVA'}, axis=1, inplace=True)
    return data[['Anio', 'Semana', 'Fecha Inicio', 'Fecha Fin', 'Cliente', 'EAN', 'Descripcion_Prod', 'Cod_Local', 'Unidades', 'Inventario', 'Precio + IVA']]

In [7]:
final = clean_data(df, date, week)

file_name = 'Alkosto' + '_' + str(date.year) + '_' + str(week.loc[0].values[0]) + '.csv'
print(file_name)
final.to_csv(path + '\\Layout\\' + file_name , index=False)

Alkosto_2021_14.csv


--------------
--------------
--------------

df = pd.DataFrame()
for file in ['Alkosto_2021_14_abr.csv', 'Alkosto_2021_14_abr.csv']:
    df = pd.concat([df, pd.read_csv(path + '\\Layout\\' + file)])

data = df.pivot_table(index=['Anio', 'Semana', 'Fecha Inicio', 'Fecha Fin', 'Cliente', 'EAN', 'Descripcion_Prod', 'Cod_Local'], values=['Unidades', 'Inventario', 'Precio + IVA'], aggfunc='sum').reset_index()

file_name = 'Alkosto' + '_' + str(date.year) + '_' + str(week.loc[0].values[0]) + '.csv'
print(file_name)
data.to_csv(path + '\\Layout\\' + file_name , index=False)